In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pint

Here the `pint` module is used because dealing with units is a real pain and this package makes them very easy to manage in python.

In [2]:
u = pint.UnitRegistry()

# Question 1

Some functions that will help us

In [3]:
def compute_I_atom(Z):
    return np.piecewise(Z,
                        [Z<=13, Z>13],
                        [lambda Z: 7+12*Z, lambda Z: 9.76+58.8*Z**(-0.19)])

def compute_I_compound(Ns, Zs):
    Is = compute_I_atom(Zs)
    numerator = sum(Ns*Zs*np.log(Is))
    denominator = sum(Ns*Zs)
    return np.exp(numerator/denominator)

## Part A

In [4]:
# Atomic Number of Hydrogen and Oxygen
Zs = np.array([1, 8])
# Atomic Weights of Hydrogen and Oxygen
Ws = np.array([1.0079, 15.999])
# Number of Species of Hydrogen and Oxygen
Ns = np.array([2, 1])

I_water_m1 = compute_I_compound(Ws*Ns, Zs)
I_water_m2 = compute_I_compound(Ns, Zs)

We can obtain an estimate using either the total atomic weight or the number of species i present in the compound:

In [5]:
print(f'Using Total Atomic Weight: I={I_water_m1:.2f}eV')
print(f'Using Number of Species: I={I_water_m2:.2f}eV')

Using Total Atomic Weight: I=100.34eV
Using Number of Species: I=73.45eV


The more accurate answer, in this case, is using the total number of species.

## Part B

In [6]:
# Atomic Number of Carbon, Hydrogen and Oxygen
Zs = np.array([6, 1, 8])
# Atomic Weights of Carbon, Hydrogen and Oxygen
Ws = np.array([12.0107, 1.0079, 15.999])
# Number of Species of Carbon, Hydrogen and Oxygen
Ns = np.array([5, 8, 2])

I_acrylic_m1 = compute_I_compound(Ws*Ns, Zs)
I_acrylic_m2 = compute_I_compound(Ns, Zs)

We can obtain an estimate using either the total atomic weight or the number of species i present in the compound:

In [7]:
print(f'Using Total Atomic Weight: I={I_acrylic_m1:.2f}eV')
print(f'Using Number of Species: I={I_acrylic_m2:.2f}eV')

Using Total Atomic Weight: I=86.47eV
Using Number of Species: I=69.20eV


## Part C

The mean ionization potential $I$ is a geometric mean of all ionization and excitation potentials of atoms of the absorbing material. It is through these absorption pathways in the material (excitiation and ionization) that an incident charged particle beam will lose energy and transfer it to the material. As such, $I$ plays a crucial role in determining the stopping power of a material, since stopping power is a measure of how a charged particle beam will deposit energy in a material. Furthermore, stopping power is a statistical average: this is precisely why **only the mean ionization potential** is required: not all excitation pathways need to be concidered in the formula. If one, however, was interested in the stochastic quantity $dE/dx$, the relative probability of individual excitiation and ionization pathways would need to be considered (and the formula, which would now be of random variables, would be significantly more complicated).

## Part D

One way to measure $I$ is to send a beam of monoenergetic charged particles with initial energy $E_0$ through a material of length $L$ and measure the outgoing energy distribution $E_f$, which will likely be narrow with a well defined mode. One can then solve the differential equation for stopping power $-\left<dE/dx\right>$ with the condition $E(x=0)=E_0$ and choose a value of $I$ that gives the closest solution to $E(x=L) = \left<E_f\right>$. This could be done through some sort of minimization algorithm. It is important to note that the energy range for the experiment should be within the valid region of the Bethe-Bloche equation.

## Part E

Acrylic has a slightly lower mean ionization potential than water: while the stopping power is thus slightly greater ($S \propto \ln(1/I)$), it is also within a similar order of magnitude:

In [8]:
np.log(1/I_water_m2)/np.log(1/I_acrylic_m2)

1.0140983305083777

so it should make a good substitute.

# Question 2

Since

$$E_k = \left(\frac{1}{\sqrt{1-\beta^2}}-1\right)mc^2$$

we have

$$\beta^2 = 1-\frac{mc^2}{(E_k+mc^2)^2}$$

In [9]:
Z = 10
NA = 6.02e23
A = 18*u.g
e = 1.602e-19 * u.C
me = 9.11e-31 * u.kg
mp = 1.67e-27 * u.kg
c = 3e8 * u.m/u.s
z = 1
eps0 = 8.854e-12 * u.F/u.m
rho = 1*u.g / u.cm**3
I = I_water_m2 * u.eV

## Part A

Use $E_k$ to get $\beta^2$

In [10]:
Ek = 100*u.MeV
beta2 = 1 - (mp*c**2 / (Ek+mp*c**2))**2

In [11]:
beta2

0.18338042882932593 <Unit('dimensionless')>

Get mass stopping power

In [12]:
C = 4*np.pi*(Z*NA/A)*(e**2 / (4*np.pi*eps0))**2 * (z**2/(me*c**2*beta2))
f = np.log(2*me*c**2/I) + np.log(beta2/(1-beta2)) -beta2
S = C*f

Print value

In [13]:
print(S.to(u.MeV*u.cm**2/u.g))

7.300673054177806 centimeter ** 2 * megaelectron_volt / gram


## Part B

In [14]:
Ek = 10*u.MeV
beta2 = 1 - (mp*c**2 / (Ek+mp*c**2))**2
C = 4*np.pi*(Z*NA/A)*(e**2 / (4*np.pi*eps0))**2 * (z**2/(me*c**2*beta2))
f = np.log(2*me*c**2/I) + np.log(beta2/(1-beta2)) -beta2
S = C*f
print(S.to(u.MeV*u.cm**2/u.g))

46.06267272747041 centimeter ** 2 * megaelectron_volt / gram


## Part C

In [15]:
Ek = 1*u.MeV
beta2 = 1 - (mp*c**2 / (Ek+mp*c**2))**2
C = 4*np.pi*(Z*NA/A)*(e**2 / (4*np.pi*eps0))**2 * (z**2/(me*c**2*beta2))
f = np.log(2*me*c**2/I) + np.log(beta2/(1-beta2)) -beta2
S = C*f
print(S.to(u.MeV*u.cm**2/u.g))

271.0776530716801 centimeter ** 2 * megaelectron_volt / gram


## Part D

According to the paper, the values are

* $E_k=100$MeV $\implies$ $S_{\text{col}}/\rho=7.25$MeV$\cdot$cm$^2$/g
* $E_k=10$MeV $\implies$ $S_{\text{col}}/\rho=45.32$MeV$\cdot$cm$^2$/g
* $E_k=1$MeV $\implies$ $S_{\text{col}}/\rho=257.4$MeV$\cdot$cm$^2$/g

The values obtained in Parts A,B,C are all within 10% of these values. Possible discrepancies may be due to using an incorrect ionization potential for water (note the true value is 78eV but the value used here is 73eV from Question 1) and discrepancies in the stopping power formula at lower energies (see the first figure here: https://en.wikipedia.org/wiki/Bethe_formula) 

# Question 3

## Part A

Refering to the diagram in Lecture 9 Slide 13, the exposure can be derived as follows. Assume that

1. $L<<f$, so that the quadrilateral EFGH is approximately a square
2. The electric field is contained mostly between the two plates (i.e. little fringing fields outside the boundaries of the plates)
3. The gap between the parallel plates is sufficiently large so that electrons liberated/created by photons **in the volume EFGH** lose all their energy (i.e. ionize as much other matter as possible) before striking the plates

If follows that the volume of air that will be ionized by the beam **with ions affected by the electric field of the capacitor** is $A_f L$ where $A_f$ is the cross sectional. The mass of air in that region is $m=\rho V=\rho A_f l$. Provided that the ionization of matter is uniform within the volume (which can be satisfied if the volume is small enough)

$$X = \frac{dQ}{dm} \approx \frac{\Delta Q}{\Delta m} = \frac{\Delta Q}{\rho A_f l}$$

## Part B

Provided one has access to the current measured as a function of time$I(t)$ (through an ammeter or some other apparatus) the charge ionized as a function of time $\Delta Q(T)$ can be measured using

$$\Delta Q(T) = \int_{t'=0}^{T} \frac{dQ}{dt}dt = \int_{t=0}^{T} I(t) dt$$

The parameter $T$ controls how long you want to measure the exposure for: clearly as the beam runs for longer and longer, the air gains a larger and larger exposure.

## Part C

We know that in air

$$K_{\text{col}} = \frac{W_{\text{air}}}{e}X$$

and at charged particle equilibrium (and approximately past that point as well), $D_{\text{med}}=K_{\text{col}}$. Furthermore, since $D_{\text{med}}=\Phi(S_{\text{col}}/\rho)$ (monoenergetic) we have

$$S_{col}/\rho = \frac{W_{\text{air}}}{\Phi e}X$$

It can be useful to write $\Phi(T) \approx N(T)/A$ where $N(T)$ is the total number of particles incident in a sphere of cross sectional area $A$ after a time $T$. As time $T$ increases, $N(T)$ gets larger and larger. It can be approximated as 

$$N(T) = \phi A \cdot \frac{vT}{2\sqrt{A/\pi}} = \frac{1}{2}\phi A^{1/2} \pi^{1/2} c \left(1-\frac{mc^2}{(E_k+mc^2)^2)}\right)^{1/2}$$

The expression above follows from:

* $n=\phi A$ is the instantaneous number of photons in a sphere of cross sectional area $A$
* $r=\sqrt{A/\pi}$ is the radius of this sphere of cross section area $A$
* $vT/2r$ gives the number of times the sphere of cross sectional area $A$ is completely replaced by new particles.
* $v=\beta c =  c \left(1-\frac{mc^2}{(E_k+mc^2)^2)}\right)^{1/2}$

If we assume that most of the stopping power is due to collisional stopping power, then

$$R = \int_{0}^{E_k} S_{col}^{-1} dE_k = \int_{0}^{E_k} \frac{N(T) e}{A \rho W_{\text{air}}}\frac{1}{X}dE_k $$

Exposure $X$ can be measured experimentally as a function of energy and time $X=X(E_k; T)$. If $R_0$ is the largest feasible range to fit in a labratory, then the highest feasible energy of beam is

$$\boxed{E_k \hspace{1mm} \text{such that} \hspace{1mm} \left(1-\frac{mc^2}{(E_k+mc^2)^2)}\right)^{-1/2} \int_{0}^{E_k} \frac{1}{X(E_k'; T)} dE_k' < \frac{\pi^{1/2} R_0 c \phi \rho W_{\text{air}}}{ 2A^{1/2}e}  }$$

where 

$$X(E_k; T) = \frac{1}{\rho A_f l} \int_{t=0}^{T} I(t;E_k) dt$$

The time $T$ can be measured for any preferred length, but the longer $T$ is, the more accurate the formula for $N(T)$ will be. So that the volumes are equal, it should also be the case that $A_f l = 4A^{3/2}3\pi^{1/2}$ (the expression on the right is the volume of a sphere written in terms of its cross sectional area).